In [10]:
import pandas as pd
from binance.client import Client
import os
import sys
import pandas_ta as ta


# read best ema cross
filename = 'coinpairBestEma.csv'
df_best_ema = pd.read_csv(filename)

api_key = os.environ.get('binance_api')
api_secret = os.environ.get('binance_secret')
client = Client(api_key, api_secret)

def get_data(coinPair, aTimeframeNum, aTimeframeTypeShort):

    try:

        lTimeFrame = str(aTimeframeNum)+aTimeframeTypeShort
        if aTimeframeTypeShort == "h":
            lTimeframeTypeLong = "hour"
        elif aTimeframeTypeShort == "d":
            lTimeframeTypeLong = "day"

        
        # if best Ema exist get price data 
        # lstartDate = str(1+gSlowMA*aTimeframeNum)+" "+lTimeframeTypeLong+" ago UTC"
        sma200 = 200
        lstartDate = str(sma200*aTimeframeNum)+" "+lTimeframeTypeLong+" ago UTC" 
        ltimeframe = str(aTimeframeNum)+aTimeframeTypeShort
        frame = pd.DataFrame(client.get_historical_klines(coinPair,
                                                        ltimeframe,
                                                        lstartDate))

        frame = frame.iloc[:,:6] # use the first 5 columns
        frame.columns = ['Time','Open','High','Low','Close','Volume'] #rename columns
        frame[['Open','High','Low','Close','Volume']] = frame[['Open','High','Low','Close','Volume']].astype(float) #cast to float
        frame['coin'] = coinPair
        frame.Time = pd.to_datetime(frame.Time, unit='ms')
        return frame
    except Exception as e:
        msg = sys._getframe(  ).f_code.co_name+" - "+coinPair+" - "+repr(e)
        print(msg)
        # telegram.send_telegram_message(telegramToken, telegram.eWarning, msg)
        frame = pd.DataFrame()
        return frame 

#-----------------------------------------------------------------------
# calculates moving averages 
#-----------------------------------------------------------------------
def apply_technicals(df):
        df['50DSMA'] = df['Close'].rolling(50).mean()
        df['200DSMA'] = df['Close'].rolling(200).mean()
        df['perc_above_200DSMA'] = ((df['Close']-df['200DSMA'])/df['200DSMA'])*100
        # df['perc_above_50DSMA'] = ((df['Close']-df['50DSMA'])/df['50DSMA'])*100


In [11]:
# coinPair = "COMPBUSD"
aTimeFrameNum = 1
atimeframeTypeShort = "d"
stablecoin = "BUSD"

try:
    exchange_info = client.get_exchange_info()
except Exception as e:
        msg = "Error connecting to Binance. "+ repr(e)
        print(msg)
        # logging.exception(msg)
        # telegram.send_telegram_message(telegram.telegramToken_market_phases, telegram.eWarning, msg)
        sys.exit(msg) 

coinPairs = set()

for s in exchange_info['symbols']:
    if (s['symbol'].endswith(stablecoin)
        and not(s['symbol'].endswith('DOWN'+stablecoin))
        and not(s['symbol'].endswith('UP'+stablecoin))
        and not(s['symbol'] == "AUD"+stablecoin) # Australian Dollar
        and not(s['symbol'] == "EUR"+stablecoin) # Euro
        and not(s['symbol'] == "GBP"+stablecoin) # British pound
        and s['status'] == 'TRADING'):
            coinPairs.add(s['symbol'])

coinPairs = sorted(coinPairs)

df_result = pd.DataFrame()

# i = 0

for coinPair in coinPairs:

    # i = i+1

    # if i == 5:
    #     break

    print("calculating "+coinPair)
    # df = pd.DataFrame()
    # print(len(df.index))
    df = get_data(coinPair, aTimeFrameNum, atimeframeTypeShort)
    apply_technicals(df)
    df.dropna(inplace=True)

    if df_result.empty:
        df_result = df
    else:
        df_result = pd.concat([df_result, df])


df_result2 = df_result[['coin','Close','50DSMA', '200DSMA', 'perc_above_200DSMA']]

# df_result.sort_values
# df = df.sort_index()
df_result2


KeyboardInterrupt: 

In [ ]:
df_sorted = df_result2.sort_values(by=['perc_above_200DSMA'], ascending=False)
df_sorted

,coin,Close,50DSMA,200DSMA,perc_above_200DSMA,perc_above_50DSMA
199,FETBUSD,0.54840,0.209566,0.114515,378.887138,161.683670
199,RNDRBUSD,2.03000,0.755020,0.592965,242.347356,168.867050
199,HIGHBUSD,4.29600,1.537380,1.472250,191.798268,179.436444
199,OCEANBUSD,0.51760,0.267220,0.191218,170.685814,93.698076
199,FISBUSD,0.86810,0.389962,0.351230,147.160332,122.611434
...,...,...,...,...,...,...
199,MCBUSD,0.39800,0.319620,0.569135,-30.069316,24.522871
199,RAYBUSD,0.29450,0.196132,0.439171,-32.941914,50.153978
199,SANTOSBUSD,5.87200,5.144880,8.882430,-33.891964,14.132886
199,SRMBUSD,0.37387,0.279685,0.571268,-34.554314,33.675480


In [ ]:
df_sorted.to_csv("most_bullish_BTC.csv")

In [ ]:
import pandas as pd

filename = 'coinpairByMarketPhase_BUSD_1d.csv'
df_top = pd.read_csv(filename)
df_top

,Unnamed: 0,Coinpair,Close,50DSMA,200DSMA,perc_above_50DSMA,perc_above_200DSMA,MarketPhase
0,13,FETBUSD,0.55100,0.209618,0.114528,162.859106,381.102957,bullish
1,35,RNDRBUSD,2.02100,0.754840,0.592920,167.738859,240.855427,bullish
2,17,HIGHBUSD,4.15400,1.534540,1.471540,170.700014,182.289302,bullish
3,30,OCEANBUSD,0.52290,0.267326,0.191244,95.603869,173.419628,bullish
4,136,MDTBUSD,0.06386,0.026196,0.026797,143.777676,138.312482,accumulation
5,7,CTXCBUSD,0.33690,0.210266,0.141577,60.225619,137.962381,bullish
6,31,OPBUSD,2.94400,1.673520,1.237345,75.916631,137.928791,bullish
7,14,FISBUSD,0.79400,0.388480,0.350859,104.386326,126.301734,bullish
8,18,INJBUSD,3.93800,1.939200,1.785020,103.073432,120.613775,bullish
9,16,FXSBUSD,13.22700,7.610540,6.172040,73.798443,114.305157,bullish


In [ ]:
import numpy as np 
df_top['Strength'] = np.arange(len(df_top))+1
df_top

,Unnamed: 0,Coinpair,Close,50DSMA,200DSMA,perc_above_50DSMA,perc_above_200DSMA,MarketPhase,strength,Strength
0,13,FETBUSD,0.55100,0.209618,0.114528,162.859106,381.102957,bullish,1,1
1,35,RNDRBUSD,2.02100,0.754840,0.592920,167.738859,240.855427,bullish,2,2
2,17,HIGHBUSD,4.15400,1.534540,1.471540,170.700014,182.289302,bullish,3,3
3,30,OCEANBUSD,0.52290,0.267326,0.191244,95.603869,173.419628,bullish,4,4
4,136,MDTBUSD,0.06386,0.026196,0.026797,143.777676,138.312482,accumulation,5,5
5,7,CTXCBUSD,0.33690,0.210266,0.141577,60.225619,137.962381,bullish,6,6
6,31,OPBUSD,2.94400,1.673520,1.237345,75.916631,137.928791,bullish,7,7
7,14,FISBUSD,0.79400,0.388480,0.350859,104.386326,126.301734,bullish,8,8
8,18,INJBUSD,3.93800,1.939200,1.785020,103.073432,120.613775,bullish,9,9
9,16,FXSBUSD,13.22700,7.610540,6.172040,73.798443,114.305157,bullish,10,10


In [ ]:
import pandas as pd

def get_performance_rank(symbol):

    if symbol.endswith("BTC"):
        coin_only = symbol[:-3]
        coin_stable = symbol[-3:]
    elif symbol.endswith(("BUSD","USDT")):    
        coin_only = symbol[:-4]
        coin_stable = symbol[-4:]

    filename = f'coinpairByMarketPhase_{coin_stable}_1d.csv'
    df = pd.read_csv(filename)

    # create column performance_rank if does not exist 
    if 'performance_rank' in df.columns:
        res = df.loc[df['Coinpair'] == symbol, 'performance_rank'].values
        if len(res) > 0:
            return res[0]
        else:
            return 1000
    else:
        return 1000

In [ ]:
symbol = "CHRBUSD"
s = get_performance_rank(symbol)
# print(s)
print(f's={s}')

s=None


In [19]:
import pandas as pd


filename = 'positions1h.csv'
df_positions = pd.read_csv(filename)
df_positions


,Currency,performance_rank,position,quantity,buyPrice,currentPrice,PnLperc
0,DASHBUSD,1000.0,1,2.945,63.140000,62.1300,-1.60
1,WOOBUSD,23.0,1,941.500,0.198618,0.2367,19.17


In [15]:
# sort positions by performance rank
df_positions.sort_values(by=['performance_rank'], inplace=True)
df_positions


KeyError: 'performance_rank'

In [ ]:
# list of coins in position - SELL
listPosition1 = df_positions[df_positions.position == 1].Currency
listPosition1

In [ ]:
# list of coins in position - BUY
listPosition0 = df_positions[df_positions.position == 0].Currency